# Correspondence-driven plane-based M3C2 (PBM3C2) with pre-segmented planes
Implemented in: py4dgeo.pbm3c2


**Related publication**
Zahs, V., Winiwarter, L., Anders, K., Williams, J.G., Rutzinger, M. & Höfle, B. (2022): Correspondence-driven plane-based M3C2 for lower uncertainty in 3D topographic change quantification. ISPRS Journal of Photogrammetry and Remote Sensing, 183, pp. 541-559. DOI: [10.1016/j.isprsjprs.2021.11.018](https://doi.org/10.1016/j.isprsjprs.2021.11.018).

## **Method description**
In this notebook, we present how the *Correspondence-driven plane-based M3C2* (PB-M3C2, [Zahs et al., 2022] algorithm for point cloud distance computation using the `py4dgeo` package.

The concept and method of PBM3C2 are explained in this scientific talk:

<a href="https://youtu.be/5pjkpajsRNU" target="_blank"><img src="https://github.com/3dgeo-heidelberg/py4dgeo/blob/main/doc/img/thumb_youtube_zahs_isprs2022.png?raw=true" alt="" width="400" /></a>

In the current implementation of PBM3C2, a plane segmentation outside py4dgeo (e.g., using CloudCompare or other tools) is required. As PB-M3C2 is a learning algorithm, it requires user-labelled input data in the process, which can be created in graphical software, such as CloudCompare.

In [ ]:
import py4dgeo
import numpy as np
import pooch
import pandas as pd
import matplotlib.pyplot as plt

In this notebook, we use a dataset of synthetic planes, which is downloaded from the py4dgeo data repository:

In [ ]:
import os
current_dir = os.getcwd()
data_path= os.path.join(os.path.dirname(current_dir), 'data')

epoch0_path = os.path.join(data_path, "plane_horizontal_t1_segmented.xyz")
epoch1_path = os.path.join(data_path, "plane_horizontal_t2_segmented.xyz")
training_segments_path = os.path.join(data_path, "testdata-labelling2.csv")

We are reading the two input epochs from XYZ files which contain a total of four columns: X, Y and Z coordinates, as well a segment ID mapping each point to a plane and normal vector components in X, Y and Z. The `read_from_xyz` functionality allows us to read additional data columns through its `additional_dimensions` parameter. It is expecting a dictionary that maps the column index to a column name.

In [ ]:
epoch0 = py4dgeo.epoch.read_from_xyz(
    epoch0_path,
    additional_dimensions={3: "segment_id"},
    delimiter=",",
)
epoch1 = py4dgeo.epoch.read_from_xyz(
    epoch1_path,
    additional_dimensions={3: "segment_id"},
    delimiter=",",
)

In [ ]:
# 检查 epoch0 和 epoch1 之间是否有相同的 segment_id（排除未分割标记 -1）

# 从 epoch 对象中提取 segment_id
ids0 = set(epoch0.additional_dimensions["segment_id"])
ids1 = set(epoch1.additional_dimensions["segment_id"])

# 排除 -1（如果需要保留也可以注释掉这两行）
ids0.discard(-1.0)
ids0.discard(-1)
ids1.discard(-1.0)
ids1.discard(-1)

shared = sorted(ids0 & ids1)

print("共有相同的 segment_id 数量：", len(shared))
if shared:
    print("相同的 segment_id（示例，最多显示100个）：", shared[:100])
else:
    print("epoch0 与 epoch1 之间没有相同的 segment_id。")

In [ ]:
ids0

In [ ]:
ids1

The point cloud data we use here consists of 100 planar segments, with 70 used for training and 30 for application.

In [ ]:
apply_ids = np.arange(1, 31)

We instantiate an instance of the algorithm class. Here, you can set the registration error for the input point clouds.

In [ ]:
alg = py4dgeo.PBM3C2(registration_error=0.01)

The algorithm requires the user to provide a labeled training dataset **correspondences_file** to learn how to match the segments. This csv file contains three columns: the first two are the plane segment_id from epoch 1 and epoch 2, and the third is a label (1 for a correct match, 0 for an incorrect one).

Here is an example of the **correspondences_file** structure:

In [ ]:
training_sample = pd.read_csv(training_segments_path, header=None, nrows=3)

print("Training correspondence file structure (first 3 rows):")
print(training_sample.to_string(index=False, header=False))
print("\nFormat explanation:")
print("  - Column 0: Segment ID from epoch 0")
print("  - Column 1: Corresponding segment ID from epoch 1")
print("  - Column 2: Label (1 = correct match, 0 = incorrect match)")

In [ ]:
correspondences_df = alg.run(
    epoch0=epoch0,
    epoch1=epoch1,
    correspondences_file=training_segments_path,
    apply_ids=apply_ids,
    search_radius=5.0,
)

In [ ]:
print(correspondences_df.head())

In [ ]:
distances = correspondences_df["distance"]
uncertainties = correspondences_df["uncertainty"]

We can visualize the matched plane correspondences and their spatial relationships:

The `visualize_correspondences` function includes parameters to control its plotting behavior, offering the following options: pinpoint a single `epoch0_segment_id` for detailed zoom-in display; enable the `show_all=True` option to render all content; or directly use the default random `num_samples` value for a quick preview.

In [ ]:
fig, ax = alg.visualize_correspondences(epoch0_segment_id=20, elev=20, azim=135)
plt.show()

## References
* Zahs, V., Winiwarter, L., Anders, K., Williams, J.G., Rutzinger, M. & Höfle, B. (2022): Correspondence-driven plane-based M3C2 for lower uncertainty in 3D topographic change quantification. ISPRS Journal of Photogrammetry and Remote Sensing, 183, pp. 541-559. DOI: [10.1016/j.isprsjprs.2021.11.018](https://doi.org/10.1016/j.isprsjprs.2021.11.018).